# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-13 21:44:35] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=39903, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=369453376, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gp

[2025-06-13 21:44:35] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-13 21:44:45] Attention backend not set. Use flashinfer backend by default.
[2025-06-13 21:44:45] Init torch distributed begin.


[2025-06-13 21:44:46] Init torch distributed ends. mem usage=0.00 GB


...

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.26s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.26s/it]

[2025-06-13 21:44:52] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=22.10 GB, mem usage=0.93 GB.
[2025-06-13 21:44:52] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-13 21:44:52] Memory pool end. avail mem=21.79 GB


[2025-06-13 21:44:52] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=22.01 GB
[2025-06-13 21:44:52] INFO:     Started server process [1121767]
[2025-06-13 21:44:52] INFO:     Waiting for application startup.
[2025-06-13 21:44:52] INFO:     Application startup complete.
[2025-06-13 21:44:52] INFO:     Uvicorn running on http://127.0.0.1:39903 (Press CTRL+C to quit)


[2025-06-13 21:44:53] INFO:     127.0.0.1:57266 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-13 21:44:53] INFO:     127.0.0.1:57278 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-13 21:44:53] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-13 21:45:57] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-13 21:46:28] INFO:     127.0.0.1:57292 - "POST /generate HTTP/1.1" 200 OK
[2025-06-13 21:46:28] The server is fired up and ready to roll!


[2025-06-13 21:46:31] INFO:     127.0.0.1:33520 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-06-13 21:46:31] Child process unexpectedly failed with exitcode=9. pid=1122756


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-13 21:46:41] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=32147, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=208173813, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_

[2025-06-13 21:46:42] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-13 21:46:53] Attention backend not set. Use flashinfer backend by default.
[2025-06-13 21:46:53] Init torch distributed begin.


[2025-06-13 21:46:53] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-13 21:46:57] Init torch distributed begin.
[2025-06-13 21:46:57] Init torch distributed ends. mem usage=0.00 GB
[2025-06-13 21:46:57] Load weight begin. avail mem=6.38 GB
[2025-06-13 21:46:57] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.35s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.35s/it]

[2025-06-13 21:46:59] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=5.21 GB, mem usage=1.17 GB.
[2025-06-13 21:46:59] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-13 21:46:59] Memory pool end. avail mem=4.89 GB


[2025-06-13 21:46:59] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=5.05 GB
[2025-06-13 21:46:59] INFO:     Started server process [1129280]
[2025-06-13 21:46:59] INFO:     Waiting for application startup.
[2025-06-13 21:46:59] INFO:     Application startup complete.
[2025-06-13 21:46:59] INFO:     Uvicorn running on http://127.0.0.1:32147 (Press CTRL+C to quit)


[2025-06-13 21:47:00] INFO:     127.0.0.1:40150 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-13 21:47:00] INFO:     127.0.0.1:40148 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-13 21:47:00] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-13 21:47:03] INFO:     127.0.0.1:40158 - "POST /generate HTTP/1.1" 200 OK
[2025-06-13 21:47:03] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-13 21:47:05] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-13 21:47:05] INFO:     127.0.0.1:40170 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-06-13 21:47:05] Child process unexpectedly failed with exitcode=9. pid=1130145


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-13 21:47:16] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=34865, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=228342103, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None

[2025-06-13 21:47:26] Casting torch.bfloat16 to torch.float16.


[2025-06-13 21:47:27] Casting torch.bfloat16 to torch.float16.
[2025-06-13 21:47:27] Attention backend not set. Use flashinfer backend by default.
[2025-06-13 21:47:27] Init torch distributed begin.


[2025-06-13 21:47:27] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-13 21:47:43] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=44.22 GB


[2025-06-13 21:47:44] INFO:     Started server process [1131834]
[2025-06-13 21:47:44] INFO:     Waiting for application startup.
[2025-06-13 21:47:44] INFO:     Application startup complete.
[2025-06-13 21:47:44] INFO:     Uvicorn running on http://127.0.0.1:34865 (Press CTRL+C to quit)


[2025-06-13 21:47:44] INFO:     127.0.0.1:44372 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-13 21:47:45] INFO:     127.0.0.1:44376 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-13 21:47:45] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-13 21:47:47] INFO:     127.0.0.1:44386 - "POST /generate HTTP/1.1" 200 OK
[2025-06-13 21:47:47] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-13 21:47:49] Detected chat template content format: string
[2025-06-13 21:47:49] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-13 21:47:50] INFO:     127.0.0.1:48730 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-06-13 21:47:50] Child process unexpectedly failed with exitcode=9. pid=1132297


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-13 21:48:00] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=32996, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=423572891, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gp

[2025-06-13 21:48:10] Casting torch.bfloat16 to torch.float16.


[2025-06-13 21:48:11] Casting torch.bfloat16 to torch.float16.
[2025-06-13 21:48:11] Attention backend not set. Use flashinfer backend by default.
[2025-06-13 21:48:11] Init torch distributed begin.


[2025-06-13 21:48:11] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-13 21:48:27] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=40.59 GB


[2025-06-13 21:48:28] INFO:     Started server process [1134237]
[2025-06-13 21:48:28] INFO:     Waiting for application startup.
[2025-06-13 21:48:28] INFO:     Application startup complete.
[2025-06-13 21:48:28] INFO:     Uvicorn running on http://127.0.0.1:32996 (Press CTRL+C to quit)


[2025-06-13 21:48:28] INFO:     127.0.0.1:40898 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-13 21:48:29] INFO:     127.0.0.1:40910 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-13 21:48:29] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-13 21:48:32] INFO:     127.0.0.1:40916 - "POST /generate HTTP/1.1" 200 OK
[2025-06-13 21:48:32] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-13 21:48:33] Detected chat template content format: string
[2025-06-13 21:48:33] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-13 21:48:33] INFO:     127.0.0.1:40932 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-06-13 21:48:33] Child process unexpectedly failed with exitcode=9. pid=1134626


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.


[2025-06-13 21:48:44] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=36703, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=232933230, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, sleep_on_idle=Fa

[2025-06-13 21:48:55] Attention backend not set. Use flashinfer backend by default.
[2025-06-13 21:48:55] Init torch distributed begin.


[2025-06-13 21:48:55] Init torch distributed ends. mem usage=0.00 GB


[2025-06-13 21:48:55] Load weight begin. avail mem=78.50 GB


...

[2025-06-13 21:49:01] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-06-13 21:49:02] INFO:     Started server process [1136197]
[2025-06-13 21:49:02] INFO:     Waiting for application startup.
[2025-06-13 21:49:02] INFO:     Application startup complete.
[2025-06-13 21:49:02] INFO:     Uvicorn running on http://0.0.0.0:36703 (Press CTRL+C to quit)


[2025-06-13 21:49:02] INFO:     127.0.0.1:56358 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-13 21:49:03] INFO:     127.0.0.1:56360 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-13 21:49:03] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-13 21:49:04] INFO:     127.0.0.1:56370 - "POST /generate HTTP/1.1" 200 OK
[2025-06-13 21:49:04] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-13 21:49:07] Detected chat template content format: string
[2025-06-13 21:49:07] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-13 21:49:08] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, accept len: 1.76, cuda graph: False, gen throughput (token/s): 10.87, #queue-req: 0


[2025-06-13 21:49:09] Decode batch. #running-req: 1, #token: 155, token usage: 0.01, accept len: 1.60, cuda graph: False, gen throughput (token/s): 103.05, #queue-req: 0


[2025-06-13 21:49:09] Decode batch. #running-req: 1, #token: 224, token usage: 0.01, accept len: 1.73, cuda graph: False, gen throughput (token/s): 108.75, #queue-req: 0


[2025-06-13 21:49:10] Decode batch. #running-req: 1, #token: 294, token usage: 0.01, accept len: 1.75, cuda graph: False, gen throughput (token/s): 111.07, #queue-req: 0


[2025-06-13 21:49:10] INFO:     127.0.0.1:47604 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-06-13 21:49:10] Child process unexpectedly failed with exitcode=9. pid=1136481


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).